In [1]:
from preproc_tabular import get_tabular
from dataset_sales import getDataset
import sales_prediction as sp
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
tab_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_anagrafica_final.csv'
aggregated_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_season.csv'
aggregated_month_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_month.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")
modello = sp.create_model(0).to(device)

batch_size = 128
epochs = 3
criterion = nn.MSELoss()
optimizer = AdamW(modello.parameters(), lr=1e-2) 
early_stop = sp.EarlyStopping(1, 0.01)

data, references, descriptions = get_tabular(aggregated_month_path)
train, val = getDataset(references, data, descriptions, img_path, batch_size)

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    sp.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2 = sp.validation_loop(val, modello, criterion, device)
    early_stp = early_stop(mse)
    print(early_stp)
    if early_stp:
        print('Early Stop attivato.')
        break
    mse_final, mae_final, r2_final = mse, mae, r2
    modello_final = modello
    
print("Done!")

os.makedirs('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\agg_m_1')
with open('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\agg_m_1\\descrizione.txt', 'w') as file:
    file.write("Addestramento con dati aggregati mensilmente, adamw, batch 128, 3 epoche, lr 1e-2, early stopping con 1 epoca e delta 0.01, dropout 0.5." + '\n\n')
    file.write(f"Metriche alla fine:\nMSE: {mse_final}\nMAE: {mae_final}\nR2: {r2_final}")
model_sc = torch.jit.script(modello_final)
model_sc.save('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\agg_m_1\\weights.pt')

c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sto usando cuda


c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Epoch 1
-------------------------------


c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


MSE: 12.167908  [  128/58058]
MSE: 6.410360  [12928/58058]
MSE: 1.895538  [25728/58058]
MSE: 0.875401  [38528/58058]
MSE: 0.390725  [51328/58058]
Validation Error: 
 Avg MSE: 0.178802 
 Avg MAE: 0.307075 
 Avg R2: 0.9755186047309484

False
Epoch 2
-------------------------------
MSE: 0.180017  [  128/58058]
MSE: 0.278931  [12928/58058]
MSE: 0.139544  [25728/58058]
MSE: 0.137787  [38528/58058]
MSE: 0.049659  [51328/58058]
Validation Error: 
 Avg MSE: 0.056505 
 Avg MAE: 0.154726 
 Avg R2: 0.9919900365364858

False
Epoch 3
-------------------------------
MSE: 0.175841  [  128/58058]
MSE: 0.151349  [12928/58058]
MSE: 0.230308  [25728/58058]
MSE: 0.117794  [38528/58058]
MSE: 0.033824  [51328/58058]
Validation Error: 
 Avg MSE: 0.041186 
 Avg MAE: 0.087096 
 Avg R2: 0.9941539706327976

False
Done!
